In [43]:
#Importing all Necessary Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import warnings
import joblib
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


Loading and Cleaning data

In [44]:
data = pd.read_csv('CLOTHING DATA.csv')
df = pd.DataFrame(data)
df

,user_id,bill_id,line_item_amount,bill_discount,transaction_date,description,inventory_category,colour,size,zone_name,store_name,year
0,519644808,741961800,559.6,0.0,2022-04-20,MBL ITA16BLT004 Regular Casual Tan 32,MENS BELT,Tan,32,North,North_6023,2022.0
1,519644808,741961800,559.6,0.0,2022-04-20,MBL ITA16BLT004 Regular Casual Tan 32,MENS BELT,Tan,32,North,North_6023,2022.0
2,504795522,692673877,699.5,0.0,2022-01-27,MBL ITA16BLT004 Regular Casual Tan 34,MENS BELT,Tan,34,East,East_6510,2022.0
3,12626591,768369011,519.6,0.0,2022-10-09,MBL ITA16BLT001 Regular Casual Dark Brown 32,MENS BELT,Dark Brown,32,South,South_6017,2022.0
4,12626591,768369011,519.6,0.0,2022-10-09,MBL ITA16BLT001 Regular Casual Dark Brown 32,MENS BELT,Dark Brown,32,South,South_6017,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1678106,420720909,756662651,1699.0,0.0,2022-08-28,MJO ITMJO00045 Jogger FF Dk Grey M. M,MJO ITMJO00045 Jogger FF Dk Grey M. M,NOT-CAPTURED,NOT-CAPTURED,South,South_7282,2022.0
1678107,420720909,756662651,1699.0,0.0,2022-08-28,MJO ITMJO00045 Jogger FF Dk Grey M. M,MJO ITMJO00045 Jogger FF Dk Grey M. M,NOT-CAPTURED,NOT-CAPTURED,South,South_7282,2022.0
1678108,420720909,756662651,1699.0,0.0,2022-08-28,MJO ITMJO00045 Jogger FF Dk Grey M. M,MJO ITMJO00045 Jogger FF Dk Grey M. M,NOT-CAPTURED,NOT-CAPTURED,South,South_7282,2022.0
1678109,448759421,773114823,1699.0,0.0,2022-10-23,MJO ITMJO00045 Jogger FF Dk Grey M. M,MJO ITMJO00045 Jogger FF Dk Grey M. M,NOT-CAPTURED,NOT-CAPTURED,East,East_7373,2022.0


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678111 entries, 0 to 1678110
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             1678111 non-null  int64  
 1   bill_id             1678111 non-null  int64  
 2   line_item_amount    1678111 non-null  float64
 3   bill_discount       1678111 non-null  float64
 4   transaction_date    1678111 non-null  object 
 5   description         1678107 non-null  object 
 6   inventory_category  1547052 non-null  object 
 7   colour              1668664 non-null  object 
 8   size                1668664 non-null  object 
 9   zone_name           1678110 non-null  object 
 10  store_name          1678110 non-null  object 
 11  year                1678110 non-null  float64
dtypes: float64(3), int64(2), object(7)
memory usage: 153.6+ MB


In [46]:
df.isnull().sum()

,0
user_id,0
bill_id,0
line_item_amount,0
bill_discount,0
transaction_date,0
description,4
inventory_category,131059
colour,9447
size,9447
zone_name,1


In [47]:
# Mode for categorical data
df['inventory_category'].fillna(df['inventory_category'].mode()[0], inplace=True)

df['colour'].fillna(df['colour'].mode()[0], inplace=True)

<ipython-input-47-35194573839e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['inventory_category'].fillna(df['inventory_category'].mode()[0], inplace=True)
<ipython-input-47-35194573839e>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].metho

In [48]:
df.isnull().sum()

,0
user_id,0
bill_id,0
line_item_amount,0
bill_discount,0
transaction_date,0
description,4
inventory_category,0
colour,0
size,9447
zone_name,1


In [49]:
# Drop rows that has missing values
df = df.dropna(subset=['description', 'size', 'zone_name', 'store_name', 'year'])

In [50]:
df.isnull().sum()

,0
user_id,0
bill_id,0
line_item_amount,0
bill_discount,0
transaction_date,0
description,0
inventory_category,0
colour,0
size,0
zone_name,0


ENCODING NON NUMERICAL

In [51]:
from sklearn.preprocessing import LabelEncoder

In [52]:
# Label Encoding
label_encoder = LabelEncoder()
df['description'] = label_encoder.fit_transform(df['description'])
df['inventory_category'] = label_encoder.fit_transform(df['inventory_category'])
df['colour'] = label_encoder.fit_transform(df['colour'])
df['zone_name'] = label_encoder.fit_transform(df['zone_name'])
df['store_name'] = label_encoder.fit_transform(df['store_name'])
df['size'] = label_encoder.fit_transform(df['size'])

<ipython-input-52-6cb76195e10b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = label_encoder.fit_transform(df['description'])
<ipython-input-52-6cb76195e10b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inventory_category'] = label_encoder.fit_transform(df['inventory_category'])
<ipython-input-52-6cb76195e10b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [53]:
df

,user_id,bill_id,line_item_amount,bill_discount,transaction_date,description,inventory_category,colour,size,zone_name,store_name,year
0,519644808,741961800,559.6,0.0,2022-04-20,8199,6527,267,9,2,37,2022.0
1,519644808,741961800,559.6,0.0,2022-04-20,8199,6527,267,9,2,37,2022.0
2,504795522,692673877,699.5,0.0,2022-01-27,8200,6527,267,10,1,3,2022.0
3,12626591,768369011,519.6,0.0,2022-10-09,8197,6527,75,9,4,89,2022.0
4,12626591,768369011,519.6,0.0,2022-10-09,8197,6527,75,9,4,89,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1678105,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0
1678106,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0
1678107,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0
1678108,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0


In [54]:
# Convert 'Date' column to datetime format
df['transaction_date'] = pd.to_datetime(df['transaction_date'], dayfirst=True, errors='coerce')

# Check if there are invalid dates
if df['transaction_date'].isnull().any():
    print("Invalid dates detected. Please check your 'transaction_date' column.")
else:
    # Convert to numerical format (YYYYMMDD)
    df['Year'] = df['transaction_date'].dt.year.astype(int)
    df['Month'] = df['transaction_date'].dt.month
    df['Day'] = df['transaction_date'].dt.day
    df = pd.get_dummies(df, columns=['Year'], drop_first=True)

<ipython-input-54-df53d335d72b>:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['transaction_date'] = pd.to_datetime(df['transaction_date'], dayfirst=True, errors='coerce')
<ipython-input-54-df53d335d72b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transaction_date'] = pd.to_datetime(df['transaction_date'], dayfirst=True, errors='coerce')
<ipython-input-54-df53d335d72b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [55]:
df

,user_id,bill_id,line_item_amount,bill_discount,transaction_date,description,inventory_category,colour,size,zone_name,store_name,year,Month,Day
0,519644808,741961800,559.6,0.0,2022-04-20,8199,6527,267,9,2,37,2022.0,4,20
1,519644808,741961800,559.6,0.0,2022-04-20,8199,6527,267,9,2,37,2022.0,4,20
2,504795522,692673877,699.5,0.0,2022-01-27,8200,6527,267,10,1,3,2022.0,1,27
3,12626591,768369011,519.6,0.0,2022-10-09,8197,6527,75,9,4,89,2022.0,10,9
4,12626591,768369011,519.6,0.0,2022-10-09,8197,6527,75,9,4,89,2022.0,10,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678105,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0,8,28
1678106,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0,8,28
1678107,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0,8,28
1678108,420720909,756662651,1699.0,0.0,2022-08-28,10539,6992,187,37,4,158,2022.0,8,28


In [56]:
# Drop the original 'Date' column
df.drop(columns=['transaction_date'], inplace=True)

In [57]:
df

,user_id,bill_id,line_item_amount,bill_discount,description,inventory_category,colour,size,zone_name,store_name,year,Month,Day
0,519644808,741961800,559.6,0.0,8199,6527,267,9,2,37,2022.0,4,20
1,519644808,741961800,559.6,0.0,8199,6527,267,9,2,37,2022.0,4,20
2,504795522,692673877,699.5,0.0,8200,6527,267,10,1,3,2022.0,1,27
3,12626591,768369011,519.6,0.0,8197,6527,75,9,4,89,2022.0,10,9
4,12626591,768369011,519.6,0.0,8197,6527,75,9,4,89,2022.0,10,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678105,420720909,756662651,1699.0,0.0,10539,6992,187,37,4,158,2022.0,8,28
1678106,420720909,756662651,1699.0,0.0,10539,6992,187,37,4,158,2022.0,8,28
1678107,420720909,756662651,1699.0,0.0,10539,6992,187,37,4,158,2022.0,8,28
1678108,420720909,756662651,1699.0,0.0,10539,6992,187,37,4,158,2022.0,8,28


SPLIT DATA INTO TESTING AND TRAINING SET

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
Y = df['inventory_category']
Y

,inventory_category
0,6527
1,6527
2,6527
3,6527
4,6527
...,...
1678105,6992
1678106,6992
1678107,6992
1678108,6992


In [60]:
X = df.drop('inventory_category',axis= 1)
X

,user_id,bill_id,line_item_amount,bill_discount,description,colour,size,zone_name,store_name,year,Month,Day
0,519644808,741961800,559.6,0.0,8199,267,9,2,37,2022.0,4,20
1,519644808,741961800,559.6,0.0,8199,267,9,2,37,2022.0,4,20
2,504795522,692673877,699.5,0.0,8200,267,10,1,3,2022.0,1,27
3,12626591,768369011,519.6,0.0,8197,75,9,4,89,2022.0,10,9
4,12626591,768369011,519.6,0.0,8197,75,9,4,89,2022.0,10,9
...,...,...,...,...,...,...,...,...,...,...,...,...
1678105,420720909,756662651,1699.0,0.0,10539,187,37,4,158,2022.0,8,28
1678106,420720909,756662651,1699.0,0.0,10539,187,37,4,158,2022.0,8,28
1678107,420720909,756662651,1699.0,0.0,10539,187,37,4,158,2022.0,8,28
1678108,420720909,756662651,1699.0,0.0,10539,187,37,4,158,2022.0,8,28


In [61]:
X_train, X_test , Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
X_train

,user_id,bill_id,line_item_amount,bill_discount,description,colour,size,zone_name,store_name,year,Month,Day
329885,492458783,702770342,719.0000,0.0,33948,187,37,3,83,2022.0,3,1
1222958,454434666,745857101,1374.3322,0.0,12978,280,35,4,152,2022.0,7,23
983457,402594927,765834381,1799.0000,0.0,31167,187,37,5,252,2022.0,10,1
1246344,511104159,716668975,2199.0000,0.0,18954,187,37,4,107,2022.0,4,20
564452,439524706,763142295,199.0000,0.0,33979,187,37,5,234,2022.0,9,21
...,...,...,...,...,...,...,...,...,...,...,...,...
260469,524087352,756566746,2499.0000,0.0,9221,187,37,4,206,2022.0,8,28
1422848,495787793,788208919,0.0020,0.0,8076,187,37,4,154,2022.0,12,9
132394,307951091,712599246,1299.0000,0.0,23531,187,37,5,244,2022.0,4,8
675470,513614266,720950047,1999.0000,0.0,27570,16,10,4,180,2022.0,5,2


In [62]:
X_test

,user_id,bill_id,line_item_amount,bill_discount,description,colour,size,zone_name,store_name,year,Month,Day
1333671,503145700,728246557,0.002000,0.000,8076,187,37,4,182,2022.0,5,26
798525,518821070,795176557,1299.500000,0.000,22382,187,37,5,248,2022.0,12,31
10600,494477009,784628116,967.595300,0.000,16076,7,39,4,100,2022.0,11,27
1614525,484575630,772221476,2599.000000,0.000,24618,187,37,4,187,2022.0,10,21
237401,519936699,779188405,1733.277527,1513.545,33948,187,37,3,83,2022.0,11,8
...,...,...,...,...,...,...,...,...,...,...,...,...
1524601,523060480,766357117,4.000000,0.000,8076,187,37,4,98,2022.0,10,2
1615208,404530889,769985079,1049.250000,3896.750,33948,187,37,3,83,2022.0,10,10
13545,518719768,741405150,683.099998,155.800,33948,187,37,3,83,2022.0,7,2
863969,460817515,753482061,2085.978000,0.000,21489,187,37,4,160,2022.0,8,17


In [39]:
# Apply SMOTE to handle class imbalance on the training set only
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train, Y_train)

# Check the distribution of classes after SMOTE
print("Class distribution after SMOTE:", pd.Series(Y_train_resampled).value_counts())

ValueError: could not convert string to float: '2022-04-10'

In [63]:
from imblearn.under_sampling import RandomUnderSampler

# Undersample the majority class
rus = RandomUnderSampler(random_state=42)
X_resampled, Y_resampled = rus.fit_resample(X, Y)

# Check new class distribution
print(pd.Series(Y_resampled).value_counts())

inventory_category
0        1
15625    1
15623    1
15622    1
15621    1
        ..
7807     1
7806     1
7805     1
7804     1
23426    1
Name: count, Length: 23427, dtype: int64


**Create and Train the Model**

In [67]:
# Create and train the XGBoost model
model = XGBClassifier(random_state=42)
model.fit(X_train, Y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [    0     1     2 ... 22881 22882 22883], got [    0     1     2 ... 23424 23425 23426]

In [66]:
#  predictions and evaluation of the the model
Y_pred = model.predict(X_test)

NotFittedError: need to call fit or load_model beforehand